In [53]:
# force kernel reload, get rid of cache
import importlib
import chromaDB_store
importlib.reload(chromaDB_store)
from chromaDB_store import ChromaDB
# setup database
db = ChromaDB("collection-test")

storing to db
adding doc 1...
adding doc 2...
adding doc 3...
adding doc 4...
adding doc 5...
adding doc 6...
adding doc 7...
adding doc 8...
adding doc 9...
adding doc 10...
adding doc 11...
adding doc 12...
adding doc 13...
adding doc 14...
adding doc 15...
adding doc 16...
adding doc 17...
adding doc 18...
adding doc 19...
adding doc 20...
adding doc 21...
adding doc 22...
adding doc 23...
adding doc 24...
adding doc 25...
adding doc 26...
adding doc 27...
adding doc 28...
adding doc 29...
adding doc 30...
adding doc 31...
adding doc 32...
adding doc 33...
adding doc 34...
adding doc 35...
adding doc 36...
adding doc 37...
adding doc 38...
adding doc 39...
adding doc 40...
adding doc 41...
adding doc 42...
adding doc 43...
adding doc 44...
adding doc 45...
adding doc 46...
adding doc 47...
adding doc 48...
adding doc 49...
adding doc 50...
adding doc 51...
adding doc 52...
adding doc 53...
adding doc 54...
adding doc 55...
adding doc 56...
adding doc 57...
adding doc 58...
adding do

In [106]:
""" This cell reads in test files & queries the LLM model with the test data """

import json
from openai_utils import analyze_results

# read in test files
large_test_data = json.load(open("test_data/expanded_test_data.json", "r"))

# analyze results from test data
combined_replies = []
for i in range(0, len(large_test_data)):
    reply = analyze_results(message=large_test_data[i]['input'], db=db)
    combined_replies.append(reply)



identified keywords: ChatCompletion(id='chatcmpl-BM6EZRXu1jRxxlIBwKHCS4kipJGsl', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='romantic, Korean drama', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, annotations=[]))], created=1744606879, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=6, prompt_tokens=140, total_tokens=146, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))




interpretted response: ChatCompletion(id='chatcmpl-BM6EaMBUhIoo1YYWpe7uMDWrmqA1m', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Looking for a romantic Korean drama with a focus on love stories and emotion

In [109]:
len(large_test_data)

26

In [107]:
""" This cell defines the function used to evaluate keywords accuracy """

from bert_score import BERTScorer
import numpy as np
import pandas as pd

def evaluate_keywords(predicted, expected):
    scorer = BERTScorer(model_type='bert-base-uncased', lang='en', rescale_with_baseline=True)
    
    pred_keywords = predicted.split(", ")
    exp_keywords = expected.split(", ")
    
    print(f"Predicted: {pred_keywords}")
    print(f"Expected: {exp_keywords}")

    # Store similarity scores (F1) in a matrix
    similarity_matrix = np.zeros((len(pred_keywords), len(exp_keywords)))

    for i, pred in enumerate(pred_keywords):
        # Duplicate the predicted keyword to match the length of expected list
        preds = [pred] * len(exp_keywords)
        # Compute score between (pred, exp[0]), (pred, exp[1]), ...
        P, R, F1 = scorer.score(preds, exp_keywords)
        similarity_matrix[i] = F1.cpu().numpy()

    # Convert to a DataFrame for readability
    df = pd.DataFrame(similarity_matrix, index=pred_keywords, columns=exp_keywords)
    print("\nBERTScore (F1) Similarity Matrix:")
    print(df.round(4))

    # average similarity
    avg_per_pred = df.mean(axis=1)
    print("\nAverage F1 per predicted keyword:")
    print(avg_per_pred.round(4))

    # Optional: aggregate stats
    max_per_pred = df.max(axis=1)
    print("\nMax F1 per predicted keyword:")
    print(max_per_pred.round(4))

    result = max_per_pred.mean()
    print(f"\nAverage Max F1 Score: {result}")
    return result

In [110]:
""" This cell evaluates the accuracy rate of genre and keywords prediction """

from tmdb_queries import get_drama_genre_id

genre_correct = 0
keywords_correct = 0
total_count = 0
data_set = large_test_data

for i in range(0, len(data_set)):
    expected_genre_id = int(data_set[i]['genre'])
    expected_keywords = data_set[i]['keywords']

    # responses for one user input
    model_responses = combined_replies[i]
    for j in range(0, len(model_responses)):
        # check if model validates response
        if model_responses[j]['is_match'].lower() == "no":
            continue

        total_count += 1
        
        # get predicted genre
        show_id = model_responses[j]['show_id']
        predicted_genre_id = get_drama_genre_id(show_id)
        
        # get predicted keywords
        predicted_keywords = model_responses[j]['keywords']

        # compare results
        if expected_genre_id in predicted_genre_id:
            genre_correct += 1
        keyword_F1 = evaluate_keywords(predicted_keywords, expected_keywords)
        if keyword_F1 >= 0.6:
            keywords_correct += 1
        

print(f"----------------------------------genre results: {genre_correct}/{total_count} = {genre_correct/total_count}")
print(f"--------------------------------keyword results: {keywords_correct}/{total_count} = {keywords_correct/total_count}")
    

Predicted: ['romantic', 'Korean drama']
Expected: ['romantic', 'Korean', 'drama', 'love', 'relationships']

BERTScore (F1) Similarity Matrix:
              romantic  Korean   drama    love  relationships
romantic        1.0000  0.3832  0.4182  0.4140         0.5844
Korean drama    0.2608  0.4283  0.4663  0.1813         0.2401

Average F1 per predicted keyword:
romantic        0.5600
Korean drama    0.3153
dtype: float64

Max F1 per predicted keyword:
romantic        1.0000
Korean drama    0.4663
dtype: float64

Average Max F1 Score: 0.7331530600786209
Predicted: ['romantic', 'Korean drama']
Expected: ['romantic', 'Korean', 'drama', 'love', 'relationships']

BERTScore (F1) Similarity Matrix:
              romantic  Korean   drama    love  relationships
romantic        1.0000  0.3832  0.4182  0.4140         0.5844
Korean drama    0.2608  0.4283  0.4663  0.1813         0.2401

Average F1 per predicted keyword:
romantic        0.5600
Korean drama    0.3153
dtype: float64

Max F1 per predic